In [2]:
import polars as pl
import pandas as pd
from ydata_profiling import ProfileReport

In [3]:
columns_id = ["id"]
columns_fact = ["loan_amnt","int_rate","num_bc_tl"]
columns_cat = ["addr_state","application_type","grade","home_ownership","initial_list_status","purpose","term"]
# columns_to_keep = columns_id + columns_fact + columns_cat
columns_to_keep =['acc_now_delinq',
 'acc_open_past_24mths',
 'addr_state',
 'all_util',
 'annual_inc',
 'annual_inc_joint',
 'application_type',
 'avg_cur_bal',
 'bc_util',
 'delinq_2yrs',
 'delinq_amnt',
 'desc',
 'dti',
 'dti_joint',
 'earliest_cr_line',
 'emp_title',
 'emp_length',
 'fico_range_high',
 'fico_range_low',
 'funded_amnt',
 'funded_amnt_inv',
 'grade',
 'home_ownership',
 'id',
 'il_util',
 'initial_list_status',
 'inq_last_12m',
 'installment',
 'int_rate',
 'issue_d',
 'last_credit_pull_d',
 'last_pymnt_d',
 'loan_amnt',
 'loan_status',
 'max_bal_bc',
 'mths_since_last_delinq',
 'mths_since_recent_bc_dlq',
 'mths_since_recent_revol_delinq',
 'num_actv_rev_tl',
 'num_rev_tl_bal_gt_0',
 'open_acc',
 'open_acc_6m',
 'open_act_il',
 'open_rv_12m',
 'open_rv_24m',
 'pct_tl_nvr_dlq',
 'percent_bc_gt_75',
 'purpose',
 'pymnt_plan',
 'revol_bal',
 'revol_util',
 'sub_grade']
path = "/home/eric/codecamp/accepted_2007_to_2018Q4.csv"
df = pl.read_csv("/home/eric/codecamp/accepted_2007_to_2018Q4.csv",ignore_errors = True, columns  = columns_to_keep, n_rows = 100000)
# df = pl.read_csv("/home/eric/codecamp/accepted_2007_to_2018Q4.csv",ignore_errors = True)

df.write_parquet("/home/eric/codecamp/accepted_2007_to_2018Q4.parquet")

df_pd = df.to_pandas()

In [ ]:
import polars as pl
import pyarrow as pa
import pyarrow.parquet as pq

path = "/home/eric/codecamp/accepted_2007_to_2018Q4.csv"
parquet_path = "/home/eric/codecamp/accepted_2007_to_2018Q4.parquet"
chunk_size = 100_000  # Adjust based on available memory

reader = pl.read_csv(path, ignore_errors=True, batch_size=chunk_size)
for i, df_chunk in enumerate(reader):
    try:
        # Ensure df_chunk is in Arrow Table format for Parquet writing
        arrow_table = pa.Table.from_pandas(df_chunk.to_pandas())
        chunk_file = f"{parquet_path}_{i}.parquet"
        pq.write_table(arrow_table, chunk_file)
    except AttributeError as e:
        print(f"Error in chunk {i}: {e}")


In [ ]:
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq

# Specify your CSV file path
csv_file = "/home/eric/codecamp/accepted_2007_to_2018Q4.csv"

# Specify Parquet output path
parquet_file = "/home/eric/codecamp/accepted_2007_to_2018Q4.parquet"

# Define chunk size (adjust according to your memory capacity)
chunk_size = 1000000  # e.g., 100k rows at a time

# Infer schema by reading the first chunk and setting the data types
first_chunk = pd.read_csv(csv_file, nrows=chunk_size, low_memory=False)
dtype_dict = {}

# Adjust dtype_dict based on first chunk
for col, dtype in first_chunk.dtypes.items():
    if dtype == "object":
        dtype_dict[col] = "string"  # Handle potential mixed types as strings
    else:
        dtype_dict[col] = dtype  # Use inferred dtype for numeric columns

# Open a Parquet writer for streaming writes
with pq.ParquetWriter(parquet_file, pa.schema(pa.Table.from_pandas(first_chunk.astype(dtype_dict)).schema), compression="snappy") as writer:
    # Process each chunk with the consistent schema
    for chunk in pd.read_csv(csv_file, chunksize=chunk_size, dtype=dtype_dict, low_memory=False):
        # Convert problematic columns to numeric, setting errors to NaN
        for col, dtype in dtype_dict.items():
            if dtype in ["int64", "float64"]:  # Adjust only numeric columns
                chunk[col] = pd.to_numeric(chunk[col], errors='coerce')
        
        # Drop rows with NaN values in any numeric columns
        chunk = chunk.dropna(subset=[col for col, dtype in dtype_dict.items() if dtype in ["int64", "float64"]])

        # Convert chunk DataFrame to an Arrow Table and write directly to Parquet
        table = pa.Table.from_pandas(chunk)
        writer.write_table(table)


In [8]:
df_sample = df_pd

In [11]:
profile = ProfileReport(df_sample, title="Profiling Report", explorative= True)

In [12]:
profile.to_file("profiling_report_explorative.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]